In [ ]:
import parameters
import numpy as np
from total_current import total_current_slm, total_current_tlm

#Define parameters class object and declare parameters
params = parameters.Params()
b = params.b
dE = params.dE
dH = params.dH
DE = params.DE
DI = params.DI
108
epsp = params.epsp
Fph = params.Fph
G0 = params.G0
LD = params.LD
N = params.N
NE = params.NE
NH = params.NH
N0 = params.N0
q = params.q
Tion = params.Tion
Vt = params.Vt

#Single-layer model dimensionalization.
def dimensionalize_slm(x_vector, t_vector, u_matrix, psi_vector):

    #The variable arrays are 2-dimensional to account for dense output.
    P = u_matrix[0:N+1,:]
    phi = u_matrix[N+1:2*N+2,:]
    n = u_matrix[2*N+2:3*N+3,:]
    p = u_matrix[3*N+3:4*N+4,:]
    x = x_vector*b
    t = t_vector*(LD*b/DI)
    P = N0*P
    phi = Vt*phi
    n = (Fph*b/DE)*n
    p = (Fph*b/DE)*p
    
#Non-Dimensional Current Density
    FP_data = total_current_slm(t_vector, u_matrix)[2]
    dd_data = total_current_slm(t_vector, u_matrix)[3]
    fn_data = total_current_slm(t_vector, u_matrix)[4]
    fp_data = total_current_slm(t_vector, u_matrix)[5]
    J_dimensional = q*G0*b*(fn_data + fp_data) - (epsp*Vt/(b*Tion))*dd_data + (q*DI*N0/b)*FP_data
    midpoint = int(np.ceil((N+1)/2))
    J_total = J_dimensional[midpoint,:]
    V_scan = psi_vector*Vt
    return x, t, P, phi, n, p, J_total, V_scan

#Three-layer model dimensionalization. """
def dimensionalize_tlm(x_vector, xE_vector, xH_vector, t_vector, u_matrix, psi_vector):

    #The variable arrays are 2-dimensional to account for dense output.
    P = u_matrix[0:N+1,:]
    phi = u_matrix[N+1:2*N+2,:]
    n = u_matrix[2*N+2:3*N+3,:]
    p = u_matrix[3*N+3:4*N+4,:]
    phiE = u_matrix[4*N+4:4*N+NE+4,:]
    nE = u_matrix[4*N+NE+4:4*N+2*NE+4,:]
    phiH = u_matrix[4*N+2*NE+4:4*N+2*NE+NH+4,:]
    pH = u_matrix[4*N+2*NE+NH+4:4*N+2*NE+2*NH+4,:]
    phi = np.concatenate((phiE, phi, phiH), 0)
    n = np.concatenate((nE, n, n[-1]*np.ones((NH,t_vector.size))), 0)
    p = np.concatenate((p[0]*np.ones((NE,t_vector.size)), p, pH), 0)
    x = np.concatenate((-xE_vector[::-1], x_vector, xH_vector), 0)*b
    t = t_vector*params.LD*params.b/params.ion_mobility_factor
    P = N0*P
    phi = Vt*phi
    n = dE*n
    p = dH*p
    
    #Non-Dimensional Current Density """
    J_total = q*G0*b*total_current_tlm(t_vector, u_matrix)[0]/10
    V_scan = psi_vector*Vt
    return x, t, P, phi, n, p, J_total, V_scan